In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from art.defences.preprocessor.gaussian_augmentation import GaussianAugmentation

gaussian_augmenter = GaussianAugmentation()

def preprocess(X,y, gaussian_augmenter):
    X, y = gaussian_augmenter(X, y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/A2PM/preprocessor/gaussian_augmenter.csv", mode='a', index=False)




In [4]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    X_test_tensor , y_test_tensor= preprocess(X_test, y_true, gaussian_augmenter)
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)

    # X_test_tensor = preprocess(X_test, gaussian_augmenter).to(device)
    # y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

    
    
    
    
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/A2PM/preprocessor/gaussian_augmenter.csv", mode='a', index=False, header=False)




In [5]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
# x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
x_train_tensor , y_train_tensor= preprocess(x_train, y_train, gaussian_augmenter)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
x_train_tensor = x_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

# x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
x_val_tensor , y_val_tensor= preprocess(x_val,y_val, gaussian_augmenter)
# y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
x_val_tensor = x_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)


train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [10]:
for epoch in range(50):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    model.eval()
    val_train_loss = 0.0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            val_train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_predictions += (predicted == labels).sum().item()

    avg_val_loss = val_train_loss / len(val_loader)
    val_accuracy = correct_predictions / len(val_dataset)

    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping check using min_delta
    if best_loss - avg_val_loss > min_delta:
        best_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 0.1256, Validation Loss: 0.1147, Validation Accuracy: 0.9650
Epoch 2, Training Loss: 0.1122, Validation Loss: 0.1124, Validation Accuracy: 0.9655
Epoch 3, Training Loss: 0.1114, Validation Loss: 0.1122, Validation Accuracy: 0.9656
Epoch 4, Training Loss: 0.1109, Validation Loss: 0.1122, Validation Accuracy: 0.9656
Epoch 5, Training Loss: 0.1106, Validation Loss: 0.1125, Validation Accuracy: 0.9657
Epoch 6, Training Loss: 0.1105, Validation Loss: 0.1118, Validation Accuracy: 0.9657
Epoch 7, Training Loss: 0.1103, Validation Loss: 0.1121, Validation Accuracy: 0.9656
Early stopping triggered


In [11]:
calculate_performance_metrics(x_test, y_test, model, 'DNN', 'Baseline', '0')

Accuracy: 0.9659648255728723

macro
Precision: 0.9212222009621819
Recall: 0.5761902024699327
F1 Score: 0.6888011381321121

weighted
Precision: 0.9649486003717612
Recall: 0.9659648255728723
F1 Score: 0.9621391948653368

Mean FNR: 0.4238097975300673
Mean TNR: 0.9148878066626717
Mean FPR: 0.08511219333732825
Mean TPR: 0.5761902024699327


In [14]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start BIM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

print("start FGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

print("start PGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

print("start DF")
# DF_eps = [0.01, 0.1, 0.2, 0.3]
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)
# calculate_performance_metrics(x_test_adv, y_test_adv, model)

start BIM
Accuracy: 0.965881056991954

macro
Precision: 0.9139508726630192
Recall: 0.5850336740013266
F1 Score: 0.6928186916653293

weighted
Precision: 0.9645378798677131
Recall: 0.965881056991954
F1 Score: 0.9622683187184561

Mean FNR: 0.41496632599867345
Mean TNR: 0.9162926932819442
Mean FPR: 0.08370730671805579
Mean TPR: 0.5850336740013266
Accuracy: 0.9572487047283176

macro
Precision: 0.5943583047930371
Recall: 0.497002247541949
F1 Score: 0.5013812897046844

weighted
Precision: 0.9539149264745601
Recall: 0.9572487047283176
F1 Score: 0.9518132880351178

Mean FNR: 0.5029977524580511
Mean TNR: 0.9007261930853872
Mean FPR: 0.0992738069146128
Mean TPR: 0.497002247541949
Accuracy: 0.9353432208181677

macro
Precision: 0.36342988694726
Recall: 0.2835183216676416
F1 Score: 0.2826997552832786

weighted
Precision: 0.9244430162769682
Recall: 0.9353432208181677
F1 Score: 0.9248138088228374

Mean FNR: 0.7164816783323584
Mean TNR: 0.8721233114289889
Mean FPR: 0.1278766885710111
Mean TPR: 0.283518

In [14]:
torch.save(model.state_dict(), "/home/jovyan/A2PM/preprocessor/dnn_pytorch_gaussian_augmenter.pt")

In [13]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BA', epsilon)

start AutoPGD
Accuracy: 0.9656088091039694

macro
Precision: 0.9326145030591217
Recall: 0.582013269587917
F1 Score: 0.6971386986269985

weighted
Precision: 0.9654644994737482
Recall: 0.9656088091039694
F1 Score: 0.9611773798376455

Mean FNR: 0.41798673041208306
Mean TNR: 0.9101481185044475
Mean FPR: 0.08985188149555248
Mean TPR: 0.582013269587917
Accuracy: 0.9624591104614393

macro
Precision: 0.8017022560316995
Recall: 0.5474855189443606
F1 Score: 0.6010346695018441

weighted
Precision: 0.9610467890121596
Recall: 0.9624591104614393
F1 Score: 0.957884352105506

Mean FNR: 0.4525144810556395
Mean TNR: 0.9085321845786124
Mean FPR: 0.09146781542138746
Mean TPR: 0.5474855189443606
Accuracy: 0.9470142783546175

macro
Precision: 0.7378890894728003
Recall: 0.4118130797003653
F1 Score: 0.42005446886216474

weighted
Precision: 0.9441629440541277
Recall: 0.9470142783546175
F1 Score: 0.9402651364314129

Mean FNR: 0.5881869202996346
Mean TNR: 0.8890847922019469
Mean FPR: 0.11091520779805306
Mean TPR

In [16]:
filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_HopSkipJump_eps_0.npy'
x_test_adv = np.load(filename)

calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'HopSkipJump', '0')

Accuracy: 0.9311003421946531

macro
Precision: 0.589002394400143
Recall: 0.38783931861583154
F1 Score: 0.41836046947185057

weighted
Precision: 0.9183724390803653
Recall: 0.9311003421946531
F1 Score: 0.9209540520855853

Mean FNR: 0.6121606813841685
Mean TNR: 0.8556589451215013
Mean FPR: 0.1443410548784987
Mean TPR: 0.38783931861583154


In [12]:
print("start ZOO")
ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]
# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'ZOO', epsilon)

start ZOO
Accuracy: 0.9656046206749235

macro
Precision: 0.9267034161937717
Recall: 0.5775072341456966
F1 Score: 0.6909287164070593

weighted
Precision: 0.9645080707261453
Recall: 0.9656046206749235
F1 Score: 0.9617179214969664

Mean FNR: 0.42249276585430345
Mean TNR: 0.9144538159286174
Mean FPR: 0.08554618407138262
Mean TPR: 0.5775072341456966
Accuracy: 0.9662663924641784

macro
Precision: 0.9245899672495893
Recall: 0.5965233091878182
F1 Score: 0.7073791961484783

weighted
Precision: 0.9652822728785947
Recall: 0.9662663924641784
F1 Score: 0.9624657292678123

Mean FNR: 0.4034766908121819
Mean TNR: 0.915240007009776
Mean FPR: 0.0847599929902239
Mean TPR: 0.5965233091878182
Accuracy: 0.9654517430147475

macro
Precision: 0.9192654183218885
Recall: 0.5873281677790783
F1 Score: 0.6996122783627376

weighted
Precision: 0.9643880122283306
Recall: 0.9654517430147475
F1 Score: 0.9615290181264927

Mean FNR: 0.4126718322209218
Mean TNR: 0.9142820426683611
Mean FPR: 0.08571795733163888
Mean TPR: 0.